# Topic Modeling with Latent Dirichlet Allocation Model
In this project extension I will explore applying an LDA model to the data. This model aims to uncover hidden structure in a collection of texts. This type of modeling can be compared to clustering (thus an interesting extension for this project) but with LDA it builds clusters of words rather than clusters of texts.  


> LDA is a generative probabilistic model that assumes each topic is a mixture over an underlying set of words, and each document is a mixture of over a set of topic probabilities.

# Libraries and Data

In [92]:
#custom functions 
from projectfunctions import * 

/Users/kristen/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [93]:
import warnings 
warnings.simplefilter('always')

import pandas as pd 
import pickle  
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np  

import gensim.corpora as corpora 

from pprint import pprint 

import pyLDAvis.gensim 
import pyLDAvis 
import os

/Users/kristen/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [94]:
#import text data
df = pickle.load( open( "Pickles/standards_corpi.pkl", "rb" ) ) 
df.drop(df.tail(1).index,inplace=True)  
df

/Users/kristen/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-94-f4e6b97e1536>:2: ResourceWarning: unclosed file <_io.BufferedReader name='Pickles/standards_corpi.pkl'>
  df = pickle.load( open( "Pickles/standards_corpi.pkl", "rb" ) )


,level_0,state,corpus
0,0,TXTfiles/alabama,"'information', 'regarding', 'course', 'study',..."
1,1,TXTfiles/alaska,"'dept', 'education', 'early', 'development', '..."
2,2,TXTfiles/arizona,"'department', 'education', 'academic', 'introd..."
3,3,TXTfiles/colorado,"'review', 'revision', 'committee', 'chairperso..."
4,4,TXTfiles/flordia,"'specifications', 'florida', 'state', 'adoptio..."
5,5,TXTfiles/georgia,"'excellence', 'first', 'excellence', 'designed..."
6,6,TXTfiles/idaho,"'content', 'state', 'superintendent', 'public'..."
7,7,TXTfiles/indiana,"'physics', 'engineering', 'process', 'seps', '..."
8,8,TXTfiles/louisiana,"'shifts', 'following', 'key', 'shifts', 'calle..."
9,9,TXTfiles/mass,"'massachusetts', 'technology', 'framework', 'e..."


# Prepare Data For LDA Analysis 

In [95]:
#set features & labels to list
corpi_list = df['corpus'].values.tolist() 
state_list = df['state'].values.tolist()

/Users/kristen/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [96]:
#clean / from strings in list 
cleaned_list = []

for word in corpi_list:  
    cleaned_string = " "
    for character in word: 
        if character.isalnum(): 
            cleaned_string += character 
        else: 
            cleaned_string += " " 
    cleaned_string = cleaned_string.split()
    cleaned_list.append(cleaned_string)

/Users/kristen/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [97]:
print(cleaned_list[:1][0][:30])

['information', 'regarding', 'course', 'study', 'materials', 'contact', 'math', 'technology', 'initiative', 'amsti', 'section', 'state', 'department', 'education', 'gordon', 'persons', 'building', 'ripley', 'street', 'montgomery', 'mail', 'box', 'montgomery', 'telephone', 'thomas', 'bice', 'state', 'superintendent', 'education', 'state']


/Users/kristen/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [98]:
#create a dictionary of words 
id2word = corpora.Dictionary(cleaned_list)  
id2word

/Users/kristen/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [50]:
#create a dictionary of words 
id2word = corpora.Dictionary(cleaned_list) 

#create corpus 
texts = cleaned_list

#TDF 
corpus = [id2word.doc2bow(text) for text in texts]

#sanity check 
print(corpus[:1][0][:30])

[(0, 8), (1, 12), (2, 6), (3, 22), (4, 1), (5, 2), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 3), (12, 3), (13, 1), (14, 1), (15, 12), (16, 6), (17, 2), (18, 1), (19, 5), (20, 1), (21, 3), (22, 4), (23, 1), (24, 1), (25, 1), (26, 3), (27, 2), (28, 2), (29, 1)]


# Train a Vanilla LDA Model 

In [90]:
#number of topics
num_topics = 12  

#build model 
lda_model = gensim.models.LdaMulticore(corpus=corpus, 
                                      id2word=id2word, 
                                      num_topics=num_topics)

#print keywords in each topic 
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

/Users/kristen/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0,
  '0.008*"evidence" + 0.008*"earth" + 0.008*"energy" + 0.008*"engineering" + '
  '0.008*"information" + 0.007*"data" + 0.006*"using" + 0.006*"ideas" + '
  '0.005*"scientific" + 0.005*"include"'),
 (1,
  '0.009*"energy" + 0.008*"scientific" + 0.006*"earth" + 0.006*"use" + '
  '0.006*"data" + 0.005*"evidence" + 0.005*"water" + 0.005*"examples" + '
  '0.005*"using" + 0.004*"different"'),
 (2,
  '0.010*"energy" + 0.008*"use" + 0.008*"information" + 0.008*"evidence" + '
  '0.007*"engineering" + 0.006*"describe" + 0.006*"scientific" + 0.006*"earth" '
  '+ 0.006*"using" + 0.006*"systems"'),
 (3,
  '0.013*"energy" + 0.009*"evidence" + 0.008*"use" + 0.007*"data" + '
  '0.007*"using" + 0.007*"earth" + 0.006*"scientific" + 0.006*"information" + '
  '0.006*"describe" + 0.005*"include"'),
 (4,
  '0.012*"energy" + 0.008*"earth" + 0.007*"scientific" + 0.007*"use" + '
  '0.006*"include" + 0.006*"evidence" + 0.006*"data" + 0.006*"engineering" + '
  '0.005*"concepts" + 0.005*"changes"'),
 (5,
  '0.

# Model Analysis 

In [91]:
#visualize topics 
pyLDAvis.enable_notebook() 

LDAvis_data_filepath= os.path.join('./results/ldavis_prepared_'+str(num_topics)) 
             
LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
              
with open('Pickles/LDA_model', 'wb') as f: pickle.dump(LDAvis_prepared, f)
             
LDAvis_prepared

/Users/kristen/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.006015  0.001115       1        1  17.907599
6      0.000723 -0.001688       2        1  17.823209
3      0.004036 -0.000460       3        1  13.649840
0      0.006707 -0.000528       4        1  12.222883
7     -0.002284  0.004159       5        1   9.542845
8      0.005986  0.002968       6        1   7.520404
2      0.001778 -0.000389       7        1   7.278434
4     -0.004168 -0.002128       8        1   6.786877
9     -0.001599  0.001867       9        1   3.176972
11     0.000657 -0.003094      10        1   2.654798
10    -0.001751 -0.002382      11        1   1.337707
1     -0.004071  0.000559      12        1   0.098431, topic_info=             Term         Freq        Total Category  logprob  loglift
1154       energy  8062.000000  8062.000000  Default  30.0000  30.0000
2926   scientific  4514.000000  4514.000000  Default  29.0000  29.0000
1059        earth  5690.000000  5690.000000  Default  28.0000  28.0000
3466          use  5007.000000  5007.000000  Default  27.0000  27.0000
837          data  4522.000000  4522.000000  Default  26.0000  26.0000
...           ...          ...          ...      ...      ...      ...
1164  engineering     3.179239  4464.425665  Topic12  -5.4550  -0.3237
900      describe     2.793256  4103.863619  Topic12  -5.5844  -0.3689
1687      include     2.739483  3967.135638  Topic12  -5.6038  -0.3545
2032       matter     2.505088  2978.661752  Topic12  -5.6933  -0.1573
1645        ideas     2.483646  2845.033730  Topic12  -5.7019  -0.1200

[1110 rows x 6 columns], token_table=       Topic      Freq         Term
term                               
15         1  0.190429     academic
15         2  0.132521     academic
15         3  0.112476     academic
15         4  0.141430     academic
15         5  0.143657     academic
...      ...       ...          ...
3602      10  0.027795        world
3602      11  0.015514        world
3602      12  0.001293        world
11396      1  0.343915  zooplankton
11396      5  0.343915  zooplankton

[5079 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 7, 4, 1, 8, 9, 3, 5, 10, 12, 11, 2])

In [ ]:
with open(LDA_data_filepath, 'rb') as f: 
             LDAvis_prepared = pickle.load(f)
             
pyLDAvis.save_html(LDAvis_prepared, ',/results/ldavis_prepared_' + 
                  str(num_topics) + '.html')

Resources: 
* [Topic Modeling in Python: Latent Dirichlet Allocation (LDA)](https://towardsdatascience.com/end-to-end-topic-modeling-in-python-latent-dirichlet-allocation-lda-35ce4ed6b3e0)